# Model evaluation

The following notebook loads the two candidate models, evaluates them using the evaluation data in the project, and stores that evaluation information in the relevant FactSheet. To use notebook, you will need the Cloud Pak for Data admin password, and the GUID of the deployment space for the models. Paste those values into the cell below, then run the cells one at a time.

In [ ]:
CPD_USERNAME = "admin"
CPD_PASSWORD = "PASSWORD"
EXPERIMENT_NAME = "AutoPolicyRiskFactSheet"
SPACE_GUID = "GUID"

Get information about the environment

In [ ]:
import os

CPD_URL = os.environ['RUNTIME_ENV_APSX_URL']
CONTAINER_ID = SPACE_GUID
CONTAINER_TYPE = "space"

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

Create a new instance of the Watson Machine Learning client.

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

Ensure that the AI FactSheets library is installed.

In [ ]:
try:
    from ibm_aigov_facts_client import AIGovFactsClient
except:
    !pip install -U ibm-aigov-facts-client
    from ibm_aigov_facts_client import AIGovFactsClient

Create a new instance of the FactSheets client, using a defined experiment name.

In [ ]:
from ibm_aigov_facts_client import AIGovFactsClient, CloudPakforDataConfig

creds=CloudPakforDataConfig(service_url=CPD_URL, username=CPD_USERNAME, password=CPD_PASSWORD, disable_ssl_verification=True)
facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

Set the deployment space with the models to the default space for the WML client, so the models can be loaded into memory.

In [ ]:
wml_client.set.default_space(SPACE_GUID)

List the available models in the space. You should see three entries here: two models, and one pipeline.

In [ ]:
wml_client.repository.list()

# ACTION: Copy and paste the model GUID values into the cell below

Copy and paste the values for the sklearn and AutoAI (P4 Ridge) model (**NOT the pipeline**) into the code below before executing the cell. When you are finished, your code should look like this:

```
sklearn_model = wml_client.repository.load('3e58af8b-8628-4796-9cb8-be2e10f9222a')
autoai_model = wml_client.repository.load('56c9c6b2-d50d-4607-8de0-cb3afa44fc6d')
```

In [ ]:
sklearn_model = wml_client.repository.load('3e58af8b-8628-4796-9cb8-be2e10f9222a')
autoai_model = wml_client.repository.load('56c9c6b2-d50d-4607-8de0-cb3afa44fc6d')

Get the evaluation data from the project, and read it into a pandas dataframe.

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

import pandas as pd

df_data_1 = pd.read_csv(wslib.mount.get_data_path('policy_risk_evaluation.csv'))
df_data_1.head()


Import two evaluation functions from the sklearn library.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

Separate the label column from the feature data, and drop the four unused columns.

In [ ]:
X = df_data_1.drop(['HOUSEHOLD_ID', 'POLICY_ID', 'LAST_NAME', 'ZIPCODE', 'RISK'], axis=1)
y = df_data_1['RISK']

Send the evaluation data to the sklearn model for scoring.

In [ ]:
sklearn_pred = sklearn_model.predict(X)

Calculate the [root mean square error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and [r2](https://en.wikipedia.org/wiki/Coefficient_of_determination) values for the sklearn algorithm.

In [ ]:
from math import sqrt

meanSquaredError = mean_squared_error(y, sklearn_pred)
rootMeanSquaredError = sqrt(meanSquaredError)
print("sklearn model RMSE (lower is better):", rootMeanSquaredError)

r2 = r2_score(y, sklearn_pred)
print("sklearn r2 score (higher is better):", r2)

Send the evaluation data to the AutoAI model for scoring.

In [ ]:
autoai_pred = autoai_model.predict(X.to_numpy())

Calculate the same two metrics for the AutoAI model.

In [ ]:
meanSquaredError = mean_squared_error(y, autoai_pred)
rootMeanSquaredError = sqrt(meanSquaredError)
print("AutoAI RMSE (lower is better):", rootMeanSquaredError)

r2 = r2_score(y, autoai_pred)
print("AutoAI r2 score (higher is better):", r2)

Send the data to the FactSheet entries for both models.